In [4]:
# ✅ Correct dataset directory and filenames
import os
import pandas as pd
import numpy as np

DATA_PATH = "/kaggle/input/fifa-23-complete-player-dataset"

def load_csv(filename, large=False, chunksize=250000):
    path = os.path.join(DATA_PATH, filename)
    print(f"\n📥 Loading {filename} ...")
    if large:
        chunks = []
        for i, chunk in enumerate(pd.read_csv(path, chunksize=chunksize, low_memory=False)):
            chunks.append(chunk)
            print(f"  → Chunk {i+1} loaded ({len(chunk)} rows)")
        df = pd.concat(chunks, ignore_index=True)
    else:
        df = pd.read_csv(path, low_memory=False)
    print(f"✅ Loaded: {filename} | Shape: {df.shape}")
    return df

In [5]:
female_coaches        = load_csv("female_coaches.csv")
female_players_legacy = load_csv("female_players (legacy).csv")
female_players        = load_csv("female_players.csv", large=True)
female_teams          = load_csv("female_teams.csv")


📥 Loading female_coaches.csv ...
✅ Loaded: female_coaches.csv | Shape: (52, 8)

📥 Loading female_players (legacy).csv ...
✅ Loaded: female_players (legacy).csv | Shape: (3196, 110)

📥 Loading female_players.csv ...
  → Chunk 1 loaded (181361 rows)
✅ Loaded: female_players.csv | Shape: (181361, 110)

📥 Loading female_teams.csv ...
✅ Loaded: female_teams.csv | Shape: (7941, 54)


In [6]:
male_coaches          = load_csv("male_coaches.csv")
male_players_legacy   = load_csv("male_players (legacy).csv")
male_players          = load_csv("male_players.csv", large=True)
male_teams            = load_csv("male_teams.csv")


📥 Loading male_coaches.csv ...
✅ Loaded: male_coaches.csv | Shape: (1155, 8)

📥 Loading male_players (legacy).csv ...
✅ Loaded: male_players (legacy).csv | Shape: (161583, 110)

📥 Loading male_players.csv ...
  → Chunk 1 loaded (250000 rows)
  → Chunk 2 loaded (250000 rows)
  → Chunk 3 loaded (250000 rows)
  → Chunk 4 loaded (250000 rows)
  → Chunk 5 loaded (250000 rows)
  → Chunk 6 loaded (250000 rows)
  → Chunk 7 loaded (250000 rows)
  → Chunk 8 loaded (250000 rows)
  → Chunk 9 loaded (250000 rows)
  → Chunk 10 loaded (250000 rows)
  → Chunk 11 loaded (250000 rows)
  → Chunk 12 loaded (250000 rows)
  → Chunk 13 loaded (250000 rows)
  → Chunk 14 loaded (250000 rows)
  → Chunk 15 loaded (250000 rows)
  → Chunk 16 loaded (250000 rows)
  → Chunk 17 loaded (250000 rows)
  → Chunk 18 loaded (250000 rows)
  → Chunk 19 loaded (250000 rows)
  → Chunk 20 loaded (250000 rows)
  → Chunk 21 loaded (250000 rows)
  → Chunk 22 loaded (250000 rows)
  → Chunk 23 loaded (250000 rows)
  → Chunk 24 load

In [7]:
def preprocess_df(df, drop_cols=None, id_col=None):
    df=df.copy()

    # basic cleaning
    df.columns = df.columns.str.strip()
    df.replace(["-", "NA", "N/A", ""], np.nan, inplace=True)

    # drop unwanted columns (URLs, images, etc.)
    if drop_cols:
        df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)

    # remove duplicates
    if id_col and id_col in df.columns:
        before = len(df)
        df.drop_duplicates(subset=[id_col], keep="last", inplace=True)
        after = len(df)
        print(f" Removed {before - after} duplicates based on '{id_col}'")
    else:
        df.drop_duplicates(inplace=True)

    # downcast numeric types
    for col in df.select_dtypes(include=["int64", "float64"]).columns:
        df[col] = pd.to_numeric(df[col], downcast="integer", errors="ignore")

    # summary
    print(f" cleaned dataframe | shape: {df.shape} | Missing: {df.isna().sum().sum()}")
    return df

In [8]:
drop_url_cols = ["player_url", "player_face_url", "coach_url", "face_url", "team_url"]

female_coaches_clean = preprocess_df(female_coaches, drop_cols=drop_url_cols, id_col="coach_id")
female_players_clean = preprocess_df(female_players, drop_cols=drop_url_cols, id_col="player_id")
female_players_legacy_clean = preprocess_df(female_players_legacy, drop_cols=drop_url_cols, id_col="player_id")
female_teams_clean = preprocess_df(female_teams, drop_cols=drop_url_cols, id_col="team_id")

male_coaches_clean = preprocess_df(male_coaches, drop_cols=drop_url_cols, id_col="coach_id")
# male_players_clean = preprocess_df(male_players, drop_cols=drop_url_cols, id_col="player_id")
male_players_legacy_clean = preprocess_df(male_players_legacy, drop_cols=drop_url_cols, id_col="player_id")
male_teams_clean = preprocess_df(male_teams, drop_cols=drop_url_cols, id_col="team_id")

 Removed 0 duplicates based on 'coach_id'
 cleaned dataframe | shape: (52, 6) | Missing: 11


/tmp/ipykernel_94/4110948774.py:23: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], downcast="integer", errors="ignore")


 Removed 179726 duplicates based on 'player_id'
 cleaned dataframe | shape: (1635, 108) | Missing: 23488
 Removed 1930 duplicates based on 'player_id'
 cleaned dataframe | shape: (1266, 108) | Missing: 17163
 Removed 7889 duplicates based on 'team_id'
 cleaned dataframe | shape: (52, 53) | Missing: 705
 Removed 0 duplicates based on 'coach_id'
 cleaned dataframe | shape: (1155, 6) | Missing: 83


/tmp/ipykernel_94/4110948774.py:23: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], downcast="integer", errors="ignore")
/tmp/ipykernel_94/4110948774.py:23: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], downcast="integer", errors="ignore")
/tmp/ipykernel_94/4110948774.py:23: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], downcast="integer", errors="ignore")
/tmp/ipykernel_94/4110948774.py:23: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly inst

 Removed 111884 duplicates based on 'player_id'
 cleaned dataframe | shape: (49699, 108) | Missing: 373009


/tmp/ipykernel_94/4110948774.py:23: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], downcast="integer", errors="ignore")


 Removed 383943 duplicates based on 'team_id'
 cleaned dataframe | shape: (1112, 53) | Missing: 12434


/tmp/ipykernel_94/4110948774.py:23: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], downcast="integer", errors="ignore")


In [15]:
def clean_large_csv(in_file, out_file, id_col, drop_cols=None, chunksize=500_000):
    """
    Read and clean a very large CSV in chunks using pandas' C engine.
    Keeps memory stable and finishes within Kaggle limits.
    """
    import pandas as pd, numpy as np, os, gc, time

    in_path = os.path.join(DATA_PATH, in_file)
    out_path = os.path.join("/kaggle/working", out_file)

    # Start fresh
    if os.path.exists(out_path):
        os.remove(out_path)

    print(f"🚀 Cleaning large file in chunks → {in_file}")
    start = time.time()
    total_rows = 0

    reader = pd.read_csv(
        in_path,
        chunksize=chunksize,
        engine="c",             # ✅ supports chunking
        low_memory=True,
        na_values=["-", "NA", "N/A", "na", ""],
    )

    for i, chunk in enumerate(reader):
        # Drop unwanted columns
        if drop_cols:
            chunk = chunk.drop(columns=[c for c in drop_cols if c in chunk.columns], errors="ignore")

        # Drop duplicates per chunk
        if id_col in chunk.columns:
            chunk = chunk.drop_duplicates(subset=[id_col])
        else:
            chunk = chunk.drop_duplicates()

        # Optimize numerics
        for col in chunk.select_dtypes(include=["int64", "float64"]).columns:
            try:
                chunk[col] = pd.to_numeric(chunk[col], downcast="integer")
            except Exception:
                pass

        # Append cleaned chunk to file
        chunk.to_csv(out_path, index=False, mode="a", header=not os.path.exists(out_path))
        total_rows += len(chunk)
        print(f"  → processed chunk {i+1}, total {total_rows:,} rows")
        del chunk
        gc.collect()

    print(f"✅ Finished {in_file} → {out_file}")
    print(f"💾 Saved to /kaggle/working | Total rows: {total_rows:,} | Time: {round(time.time()-start, 2)}s")

In [16]:
drop_url_cols = ["player_url", "player_face_url", "coach_url", "face_url", "team_url"]

clean_large_csv(
    "male_players.csv",
    "male_players_clean.csv",
    id_col="player_id",
    drop_cols=drop_url_cols,
    chunksize=750_000  # safe and much faster
)

🚀 Cleaning large file in chunks → male_players.csv
  → processed chunk 1, total 26,164 rows
  → processed chunk 2, total 51,067 rows
  → processed chunk 3, total 73,139 rows
  → processed chunk 4, total 98,181 rows
  → processed chunk 5, total 122,753 rows
  → processed chunk 6, total 143,375 rows
  → processed chunk 7, total 166,949 rows
  → processed chunk 8, total 187,670 rows
  → processed chunk 9, total 210,604 rows
  → processed chunk 10, total 229,625 rows
  → processed chunk 11, total 252,655 rows


/tmp/ipykernel_94/2093646256.py:27: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):


  → processed chunk 12, total 273,140 rows


/tmp/ipykernel_94/2093646256.py:27: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):


  → processed chunk 13, total 296,621 rows


/tmp/ipykernel_94/2093646256.py:27: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):


  → processed chunk 14, total 314,486 rows
✅ Finished male_players.csv → male_players_clean.csv
💾 Saved to /kaggle/working | Total rows: 314,486 | Time: 168.42s


In [17]:
import os
os.listdir("/kaggle/working")

['.virtual_documents', 'male_players_clean.csv']

In [18]:
# Save all cleaned smaller DataFrames to working directory
datasets = {
    "female_coaches_clean": female_coaches,
    "female_players_clean": female_players,
    "female_players_legacy_clean": female_players_legacy,
    "female_teams_clean": female_teams,
    "male_coaches_clean": male_coaches,
    "male_players_legacy_clean": male_players_legacy,
    "male_teams_clean": male_teams
}

for name, df in datasets.items():
    path = f"/kaggle/working/{name}.csv"
    df.to_csv(path, index=False)
    print(f"✅ Saved {name}.csv → /kaggle/working/")

✅ Saved female_coaches_clean.csv → /kaggle/working/
✅ Saved female_players_clean.csv → /kaggle/working/
✅ Saved female_players_legacy_clean.csv → /kaggle/working/
✅ Saved female_teams_clean.csv → /kaggle/working/
✅ Saved male_coaches_clean.csv → /kaggle/working/
✅ Saved male_players_legacy_clean.csv → /kaggle/working/
✅ Saved male_teams_clean.csv → /kaggle/working/


In [19]:
import os
os.listdir("/kaggle/working")

['female_coaches_clean.csv',
 'female_players_clean.csv',
 'male_coaches_clean.csv',
 'female_teams_clean.csv',
 '.virtual_documents',
 'male_players_legacy_clean.csv',
 'male_teams_clean.csv',
 'female_players_legacy_clean.csv',
 'male_players_clean.csv']